In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install oletools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.2/977.2 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 50.3 MB/s eta 0:00:00
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35432 sha256=1e0f75dc5d85406d56001dd89a52e736f2c9e7c22cd9008de62f77183fe8b642
  Stored in directory: /root/.cache/pip/wheels/64/b8/ba/ebba30390fbd997074f35e42a842ce3fd933213cac8753414e
Successfully built olefile
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9


In [ ]:
import pandas as pd

# Define the path to the CSV file in your Google Drive
csv_path = '/content/all_results.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Convert the DataFrame to a list (and flatten it)

all_results = [item for sublist in df.values.tolist() for item in sublist]
len(all_results)

1316

In [ ]:
import pandas as pd

# Define the path to the CSV file in your Google Drive
csv_path = '/content/results_macro_sus.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Convert the DataFrame to a list (and flatten it)

all_results_sus = [item for sublist in df.values.tolist() for item in sublist]
len(all_results_sus)

179

In [ ]:
results_safe = list(set(all_results) - set(all_results_sus))
len(results_safe)

1137

#Let's be sure our files are f**king safe.

In [ ]:
from requests.exceptions import RequestException
import requests
import tempfile
from io import BytesIO
import os
import oletools
from oletools import olevba
import subprocess

count = 0
show_results = 0
results_macro_safe = []

for url in results_safe:
    try:
        response = requests.get(url, timeout=5)
    except RequestException:
        pass

    if show_results % 25 == 0:
        print("gabbogabri", url)

    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(response.content)
        temp_file_path = temp_file.name

    # Check for macros using olevba
    vba = olevba.VBA_Parser(temp_file_path)
    if vba.detect_vba_macros():
        print(f"Document {url} has macros embedded in it.")
        # Scan the VBA code using mraptor for flagged malicious macros
        mraptor_result = subprocess.run(["mraptor", temp_file_path], capture_output=True)

        if mraptor_result.returncode == 20:
            print(f"Document {url} contains potentially malicious VBA code:\n{mraptor_result.stdout.decode()}")
        else:
          #save the safe macro embedded files
            print(f"Document {url} does not contain potentially malicious VBA code.")
            results_macro_safe.append(url)

    #ignore the macro-less files
    else:
        count += 1
        print(f"{count} deleted urls.")

    show_results += 1
    

    # Clean up the temp file
    os.remove(temp_file_path)

gabbogabri http://teachmanchester.com/wp_staging/wp-content/uploads/2016/12/SLE_application_form_Dec_16.docm
Document http://teachmanchester.com/wp_staging/wp-content/uploads/2016/12/SLE_application_form_Dec_16.docm has macros embedded in it.
Document http://teachmanchester.com/wp_staging/wp-content/uploads/2016/12/SLE_application_form_Dec_16.docm does not contain potentially malicious VBA code.
Document https://e.fbr.gov.pk/SOP/IRIS/Withholding_Tax_Statement.xlsm has macros embedded in it.
Document https://e.fbr.gov.pk/SOP/IRIS/Withholding_Tax_Statement.xlsm does not contain potentially malicious VBA code.
Document https://resources.finalsite.net/files/v1634322627/amsacsorg/ajollyya3d5cdbg1xx26/ESSERIIIImplementationPlan.xlsm has macros embedded in it.
Document https://resources.finalsite.net/files/v1634322627/amsacsorg/ajollyya3d5cdbg1xx26/ESSERIIIImplementationPlan.xlsm does not contain potentially malicious VBA code.
Document https://www.gs1.org/sites/default/files/gdm/GDM_r1.2_att

Document https://www.michigan.gov/mshda/-/media/Project/Websites/mshda/rental/Property-Managers/Management-Agents/AM-Policies-and-Forms/Section-8-Utility-Rent-Schedule-Template.xlsm%3Frev%3Db1f383f5cb734e2f988f13eb441a5fca%26hash%3D5EB6C573468CD95521271A1231977707 does not contain potentially malicious VBA code.
Document https://198.1.123.167/nevadacomparecare/downloads/nv-reports/CY2016/hospitals/Centennial%2520Hills%2520NHR%25202016.xlsm has macros embedded in it.
Document https://198.1.123.167/nevadacomparecare/downloads/nv-reports/CY2016/hospitals/Centennial%2520Hills%2520NHR%25202016.xlsm does not contain potentially malicious VBA code.
Document https://www.bazl.admin.ch/dam/bazl/de/dokumente/Fachleute/Ausbildung_und_Lizenzen/sbfl/TemplatesDTO/dtotemplatesafetypolicyattachment.docm.download.docm/dto_template_safetypolicy_attachment.docm has macros embedded in it.
Document https://www.bazl.admin.ch/dam/bazl/de/dokumente/Fachleute/Ausbildung_und_Lizenzen/sbfl/TemplatesDTO/dtotemplat

In [ ]:
len(results_macro_safe)

1008

In [ ]:
import json
import pandas as pd

# Write results to JSON file
with open("results_macro_safe.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(results_macro_safe, f, indent=2) 

# Read results from JSON file and convert to DataFrame
with open("results_macro_safe.json", 'r') as f:
    results_macro_safe = json.load(f)
df = pd.DataFrame(results_macro_safe)

# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/Scraped_Links'

# Save the DataFrame as a CSV file in your Google Drive
csv_path = folder_path + '/results_macro_safe.csv'
df.to_csv(csv_path, index=False)

print('CSV file saved to:', csv_path)

CSV file saved to: /content/drive/MyDrive/Scraped_Links/results_macro_safe.csv


#VBA Macro search

In [12]:
import requests
from requests.exceptions import RequestException
import tempfile
from oletools import olevba
import os

macro_count = []
print_100 = 100
for url in results_macro_safe:

  try:
        response = requests.get(url, timeout=5)
  except RequestException:
        pass

  with tempfile.NamedTemporaryFile(delete=False) as temp_file:
      temp_file.write(response.content)
      temp_file_path = temp_file.name
  
  if print_100 % 100 == 0:
    print("gabbogabri")
    print(url, macro_count)

  # Check for macros using olevba
  vba = olevba.VBA_Parser(temp_file_path)
  if vba.detect_vba_macros():
      count_macro = 0
      #print(f"Document {url} has macros embedded in it.")
      macros = vba.extract_macros()
      for name, oledir, _, code in macros:
      #    print(f"Macro '{name}' in directory '{oledir}':\n{code}")
            count_macro+=1
      macro_count.append(count_macro)
      #print(url, count_macro)
  else:
    print("scemo")
  
  print_100 +=1

  # Clean up the temp file
  os.remove(temp_file_path)

gabbogabri
http://teachmanchester.com/wp_staging/wp-content/uploads/2016/12/SLE_application_form_Dec_16.docm []


ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 2094, in _extract_vba
    project = VBA_Project(ole, vba_root, project_path, dir_path, relaxed)
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 1752, in __init__
    projectdocstring_id = struct.unpack("<H", dir_stream.read(2))[0]
struct.error: unpack requires a buffer of 2 bytes
ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/

gabbogabri
https://doh.wa.gov/sites/default/files/2022-05/348-650-WAHL7ImplementationGuide.xlsm%3F... [1, 1, 16, 15, 1, 1, 3, 25, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 12, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 8, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 1, 1, 1, 2, 1, 1, 15, 15, 15, 1, 1, 1, 1, 1, 1, 62, 1, 1, 63, 63, 63, 1, 308, 1, 1, 7, 1, 1, 51, 1, 1, 1, 1, 1, 1, 13, 17, 7, 1, 1, 5, 1, 5, 5, 1, 1, 1, 1, 24, 1, 1, 5, 1]


ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 2094, in _extract_vba
    project = VBA_Project(ole, vba_root, project_path, dir_path, relaxed)
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 1752, in __init__
    projectdocstring_id = struct.unpack("<H", dir_stream.read(2))[0]
struct.error: unpack requires a buffer of 2 bytes
ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/

gabbogabri
https://www.cnesst.gouv.qc.ca/sites/default/files/documents/formulaire-pafisst-2020.xlsm [1, 1, 16, 15, 1, 1, 3, 25, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 12, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 8, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 1, 1, 1, 2, 1, 1, 15, 15, 15, 1, 1, 1, 1, 1, 1, 62, 1, 1, 63, 63, 63, 1, 308, 1, 1, 7, 1, 1, 51, 1, 1, 1, 1, 1, 1, 13, 17, 7, 1, 1, 5, 1, 5, 5, 1, 1, 1, 1, 24, 1, 1, 5, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 1, 12, 6, 1, 1, 1, 20, 1, 10, 41, 6, 1, 1, 9, 29, 20, 1, 4, 4, 1, 14, 1, 1, 1, 62, 9, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 6, 1, 1, 1, 8, 1, 1, 1, 1, 8, 36, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 44, 1, 1, 1, 1, 1, 8, 1, 6, 9, 1, 1, 15, 1, 2, 1, 1, 1, 10, 20, 1, 1, 1, 5, 1, 9, 1, 1, 1, 2, 36, 19, 16]


...

gabbogabri
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/536405/use-of-dhps-end-year-publication-underlying-data-2015-16.xlsm [1, 1, 16, 15, 1, 1, 3, 25, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 12, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 8, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 1, 1, 1, 2, 1, 1, 15, 15, 15, 1, 1, 1, 1, 1, 1, 62, 1, 1, 63, 63, 63, 1, 308, 1, 1, 7, 1, 1, 51, 1, 1, 1, 1, 1, 1, 13, 17, 7, 1, 1, 5, 1, 5, 5, 1, 1, 1, 1, 24, 1, 1, 5, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 1, 12, 6, 1, 1, 1, 20, 1, 10, 41, 6, 1, 1, 9, 29, 20, 1, 4, 4, 1, 14, 1, 1, 1, 62, 9, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 6, 1, 1, 1, 8, 1, 1, 1, 1, 8, 36, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 44, 1, 1, 1, 1, 1, 8, 1, 6, 9, 1, 1, 15, 1, 2, 1, 1, 1, 10, 20, 1, 1, 1, 5, 1, 9, 1, 1, 1, 2, 36, 19, 16, 1, 12, 1, 8, 1, 1, 10, 8, 1, 3, 2, 12, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 27, 1, 1, 6, 1, 11, 1, 1, 1, 1, 1, 5, 1, 1, 1, 8, 1, 8, 1, 12, 1, 1, 1, 3, 3, 1, 1, 7, 1, 1, 23, 2, 1, 5, 1, 42, 

ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 2094, in _extract_vba
    project = VBA_Project(ole, vba_root, project_path, dir_path, relaxed)
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 1752, in __init__
    projectdocstring_id = struct.unpack("<H", dir_stream.read(2))[0]
struct.error: unpack requires a buffer of 2 bytes
ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/

gabbogabri
https://www.erstrategies.org/cms/files/3610-ss-school-scheduling-master-template-new.xlsm [1, 1, 16, 15, 1, 1, 3, 25, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 12, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 8, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 1, 1, 1, 2, 1, 1, 15, 15, 15, 1, 1, 1, 1, 1, 1, 62, 1, 1, 63, 63, 63, 1, 308, 1, 1, 7, 1, 1, 51, 1, 1, 1, 1, 1, 1, 13, 17, 7, 1, 1, 5, 1, 5, 5, 1, 1, 1, 1, 24, 1, 1, 5, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 1, 12, 6, 1, 1, 1, 20, 1, 10, 41, 6, 1, 1, 9, 29, 20, 1, 4, 4, 1, 14, 1, 1, 1, 62, 9, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 6, 1, 1, 1, 8, 1, 1, 1, 1, 8, 36, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 44, 1, 1, 1, 1, 1, 8, 1, 6, 9, 1, 1, 15, 1, 2, 1, 1, 1, 10, 20, 1, 1, 1, 5, 1, 9, 1, 1, 1, 2, 36, 19, 16, 1, 12, 1, 8, 1, 1, 10, 8, 1, 3, 2, 12, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 27, 1, 1, 6, 1, 11, 1, 1, 1, 1, 1, 5, 1, 1, 1, 8, 1, 8, 1, 12, 1, 1, 1, 3, 3, 1, 1, 7, 1, 1, 23, 2, 1, 5, 1, 42, 1, 19, 5, 1, 1, 1, 1, 1, 23, 5, 1, 1, 1, 1, 11, 1, 83, 27, 1, 310, 310, 3, 

ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 2094, in _extract_vba
    project = VBA_Project(ole, vba_root, project_path, dir_path, relaxed)
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 1752, in __init__
    projectdocstring_id = struct.unpack("<H", dir_stream.read(2))[0]
struct.error: unpack requires a buffer of 2 bytes
ERROR:olevba:PROJECTNAME_SizeOfProjectName value not in range [1-128]: 131075
ERROR:olevba:Error in _extract_vba
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/oletools/olevba.py", line 3526, in extract_macros
    for stream_path, vba_filename, vba_code in \
  File "/usr/local/lib/python3.9/

gabbogabri
https://files.asprtracie.hhs.gov/documents/aspr-tracie-healthcare-coalition-resource-and-gap-analysis-aggregator.xlsm [1, 1, 16, 15, 1, 1, 3, 25, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 12, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 8, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 1, 1, 1, 2, 1, 1, 15, 15, 15, 1, 1, 1, 1, 1, 1, 62, 1, 1, 63, 63, 63, 1, 308, 1, 1, 7, 1, 1, 51, 1, 1, 1, 1, 1, 1, 13, 17, 7, 1, 1, 5, 1, 5, 5, 1, 1, 1, 1, 24, 1, 1, 5, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 1, 12, 6, 1, 1, 1, 20, 1, 10, 41, 6, 1, 1, 9, 29, 20, 1, 4, 4, 1, 14, 1, 1, 1, 62, 9, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 6, 1, 1, 1, 8, 1, 1, 1, 1, 8, 36, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 44, 1, 1, 1, 1, 1, 8, 1, 6, 9, 1, 1, 15, 1, 2, 1, 1, 1, 10, 20, 1, 1, 1, 5, 1, 9, 1, 1, 1, 2, 36, 19, 16, 1, 12, 1, 8, 1, 1, 10, 8, 1, 3, 2, 12, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 27, 1, 1, 6, 1, 11, 1, 1, 1, 1, 1, 5, 1, 1, 1, 8, 1, 8, 1, 12, 1, 1, 1, 3, 3, 1, 1, 7, 1, 1, 23, 2, 1, 5, 1, 42, 1, 19, 5, 1, 1, 1, 1, 1, 23, 5, 1, 1, 1, 1, 11,

Non sta uscendo nessuno "scemo" so far. I 1008 documenti sembrano effettivamente legittimi.

In [13]:
total = 0
for i in macro_count:
  total += i
total

12614

In [14]:
len(macro_count)

1008

In [15]:
# Let's create a list using results_macro_safe as the first column and macro_count as the second column

df = pd.DataFrame({'Column1': results_macro_safe, 'Column2': macro_count})
print(df)

                                                Column1  Column2
0     http://teachmanchester.com/wp_staging/wp-conte...        1
1     https://e.fbr.gov.pk/SOP/IRIS/Withholding_Tax_...        1
2     https://resources.finalsite.net/files/v1634322...       16
3     https://www.gs1.org/sites/default/files/gdm/GD...       15
4     https://www.floridajobs.org/docs/default-sourc...        1
...                                                 ...      ...
1003  https://multimedia-ext.bnf.fr/docs/Observatoir...        1
1004  https://iowadot.gov/design/ShellLetters/Projec...        1
1005  https://www.oracle.com/webfolder/technetwork/d...        1
1006  https://w3.blaby.gov.uk/decision-making/docume...        2
1007  https://www.azed.gov/sites/default/files/2022/...        1

[1008 rows x 2 columns]


In [16]:
results_macro_safe_count = df

In [18]:
# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/Scraped_Links'

# Save the DataFrame as a CSV file in your Google Drive
csv_path = folder_path + '/results_macro_safe_count.csv'
df.to_csv(csv_path, index=False)

print('CSV file saved to:', csv_path)

CSV file saved to: /content/drive/MyDrive/Scraped_Links/results_macro_safe_count.csv
